In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
! ls

drive  sample_data


In [9]:
from scipy.spatial import distance as dist
from imutils import face_utils
import imutils
import dlib
import cv2
import os
import numpy as np
import pandas as pd


#Calculating Distances from d1 to d8

def distance(mouth, nose, lefteye, righteye):
 d1 = dist.euclidean(mouth[0], mouth[6])
 d2 = dist.euclidean(mouth[3], mouth[9])
 d3 = dist.euclidean(mouth[0], nose[4])
 d4 = dist.euclidean(mouth[6], nose[8])
 d5 = dist.euclidean(mouth[0], righteye[0])
 d6 = dist.euclidean(mouth[6], lefteye[3])
 d7 = dist.euclidean(mouth[2], mouth[10])
 d8 = dist.euclidean(mouth[4], mouth[8])

 return d1,d2,d3,d4,d5,d6,d7,d8

shape_predictor = '/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor)

(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS['mouth']
(nStart, nEnd) = face_utils.FACIAL_LANDMARKS_IDXS['nose']
(eLStart, eLEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(eRStart, eREnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']


directory = '/content/drive/MyDrive/RefinedDataSet'
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        total= directory+'/'+filename
        frame=cv2.imread(total)
        frame=imutils.resize(frame, width=450)
        gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects=detector(gray, 0)
        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            mouth = shape[mStart:mEnd]
            nose = shape[nStart:nEnd]
            lefteye = shape[eLStart:eLEnd]
            rightteye = shape[eRStart:eREnd]
            array=[distance(mouth, nose, lefteye, rightteye)]
          #  print(array)
          #Save Distances into CSV File
            results=pd.DataFrame(array)
            results.to_csv('result6.csv',mode='a',header='True',index='false')

In [10]:

			#RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler 

path =('/content/drive/MyDrive/result.csv') #CSV PATH
dataset =pd.read_csv(path)

X = dataset.iloc[:,1:8]  #X equal to all distances
y = dataset.iloc[:, 0].values    #Y equal to gender

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

sc= StandardScaler()   #DataPreProcessing Standarlization 
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

classifier = RandomForestClassifier(n_estimators =50,random_state= 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[5 1]
 [2 4]]
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.83      0.77         6
           1       0.80      0.67      0.73         6

    accuracy                           0.75        12
   macro avg       0.76      0.75      0.75        12
weighted avg       0.76      0.75      0.75        12

Accuracy: 0.75
